In [17]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [18]:
# original_df = pd.read_excel('./main_df_dropna_completed.xlsx',index_col=0)

In [19]:
# original_df.drop(['K2_부실판단'],axis=1,inplace=True)

In [20]:
# df_train = original_df.loc[original_df['회계년도'] <= 2019]
# df_test = original_df.loc[original_df['회계년도'] > 2019]

In [21]:
# X_train = df_train.drop(['Target','회사명', '거래소코드', '회계년도'], axis=1)
# y_train = df_train['Target']

# X_test = df_test.drop(['Target','회사명', '거래소코드', '회계년도'],axis=1)
# y_test = df_test['Target']

In [22]:
df_train=pd.read_excel('./train.xlsx',index_col=0)
df_test = pd.read_excel('./test.xlsx',index_col=0)

In [23]:
df_train = df_train.reset_index()
df_test = df_test.reset_index()

In [24]:
df_train = df_train.drop(['index','회사명', '거래소코드', '회계년도','K2_부실판단', '산업위험_경기민감도','주거래은행'],axis=1)
df_test = df_test.drop(['index','회사명', '거래소코드', '회계년도','K2_부실판단', '산업위험_경기민감도','주거래은행'],axis=1)

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
selected_columns =  ['유동비율(%)', '당좌비율(%)', '비유동비율(%)', '부채비율(%)', '유동부채비율(%)', '차입금의존도(%)',
    '이익잉여금비율(%)', '순운전자본비율(%)', '비유동장기적합률(%)', '이자보상배율(이자비용)(배)',
    '외화포지션(배)', '총자본순이익률(%)', '자기자본순이익률(%)', '경영자본순이익률(%)', '매출액순이익률(%)',
    '총자본정상영업이익률(%)', '자기자본정상영업이익률(%)', '매출액정상영업이익률(%)', '금융비용부담률(%)',
    '총자산회전율(배)', '매출채권회전률(배)', '당좌자산회전률(배)', '재고자산회전률(배)', '유형자산회전율(배)',
    '매입채무회전률(배)', '자기자본회전률(배)', '경영자본회전률(배)', '유동자산증가율(%)', '매출액증가율(%)',
    '정상영업이익증가율(%)', '순이익증가율(%)', 'spread(%)', 'PPI(2015기준)', '실질GDP성장률(%)',
    'EV/EBITDA(배)', '52주베타(배)', 'PER', 'PBR', 'PCR', 'CASH FLOW 대 부채비율(%)',
    '총자본투자효율(%)', 'log_평균총자산', 'K2_score']# 수치형 데이터 컬럼의 인덱스를 넣어주세요.

# StandardScaler 객체를 생성합니다.
scaler = StandardScaler()

# 데이터프레임의 선택된 수치형 데이터 컬럼에 스탠다드 스케일링을 적용하고, 새로운 컬럼에 저장합니다.
df_train_2 = df_train .copy()  # 원본 데이터프레임을 복사합니다.
df_train_2[selected_columns] = scaler.fit_transform(df_train [selected_columns])
df_test_2 = df_test .copy()  # 원본 데이터프레임을 복사합니다.
df_test_2[selected_columns] = scaler.transform(df_test[selected_columns])


In [27]:
X_train = df_train_2[['순운전자본비율(%)','총자본순이익률(%)','자기자본정상영업이익률(%)','총자산회전율(배)','52주베타(배)','최대주주특수관계인_지분율(%)','업력(년)', '평균근속연수(년)', '직원수증감율(%)',
       '기간제비율(%)','R&D집중도(%)','K2_score','매출액순이익률(%)']]
y_train = df_train['Target']

X_test = df_test_2[['순운전자본비율(%)','총자본순이익률(%)','자기자본정상영업이익률(%)','총자산회전율(배)','52주베타(배)','최대주주특수관계인_지분율(%)','업력(년)', '평균근속연수(년)', '직원수증감율(%)',
       '기간제비율(%)','R&D집중도(%)','K2_score','매출액순이익률(%)']]
y_test = df_test['Target']


In [28]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=0,sampling_strategy=1)
x_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

In [29]:
X_train_final = df_train_2.drop("Target",axis=True)
y_train_fianl = df_train_2['Target']

In [30]:
df_train_2 = pd.concat([x_train_smote, y_train_smote],axis=1)

In [31]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

val_accuracy_list = []
val_precision_list = []
val_recall_list = []
val_f1_list = []

In [32]:
for train_idx, val_idx in kf.split(X_train):
    X_train_fold, X_val_fold = X_train_final .iloc[train_idx], X_train_final .iloc[val_idx]
    y_train_fold, y_val_fold = y_train_fianl.iloc[train_idx], y_train_fianl.iloc[val_idx]

    # CatBoostClassifier 정의
    model = CatBoostClassifier()

    # 모델 학습 (cat_features로 범주형 변수 설정)
    model.fit(X_train_fold, y_train_fold)

    # Validation 데이터로 모델 검증
    y_val_pred = model.predict(X_val_fold)
    val_accuracy_list.append(accuracy_score(y_val_fold, y_val_pred))
    val_precision_list.append(precision_score(y_val_fold, y_val_pred))
    val_recall_list.append(recall_score(y_val_fold, y_val_pred))
    val_f1_list.append(f1_score(y_val_fold, y_val_pred))

Learning rate set to 0.021977
0:	learn: 0.6597395	total: 20.9ms	remaining: 20.9s
1:	learn: 0.6271244	total: 40.8ms	remaining: 20.4s
2:	learn: 0.5982394	total: 59.1ms	remaining: 19.6s
3:	learn: 0.5742539	total: 79.1ms	remaining: 19.7s
4:	learn: 0.5485718	total: 101ms	remaining: 20.1s
5:	learn: 0.5242143	total: 119ms	remaining: 19.8s
6:	learn: 0.5034857	total: 133ms	remaining: 18.8s
7:	learn: 0.4810485	total: 147ms	remaining: 18.2s
8:	learn: 0.4609366	total: 159ms	remaining: 17.5s
9:	learn: 0.4430003	total: 170ms	remaining: 16.8s
10:	learn: 0.4268630	total: 183ms	remaining: 16.5s


11:	learn: 0.4101819	total: 197ms	remaining: 16.2s
12:	learn: 0.3950353	total: 213ms	remaining: 16.2s
13:	learn: 0.3810184	total: 230ms	remaining: 16.2s
14:	learn: 0.3702350	total: 244ms	remaining: 16s
15:	learn: 0.3577583	total: 256ms	remaining: 15.7s
16:	learn: 0.3466020	total: 267ms	remaining: 15.5s
17:	learn: 0.3362825	total: 278ms	remaining: 15.2s
18:	learn: 0.3260897	total: 291ms	remaining: 15s
19:	learn: 0.3170017	total: 301ms	remaining: 14.8s
20:	learn: 0.3087996	total: 313ms	remaining: 14.6s
21:	learn: 0.3007653	total: 330ms	remaining: 14.7s
22:	learn: 0.2923631	total: 343ms	remaining: 14.6s
23:	learn: 0.2855022	total: 357ms	remaining: 14.5s
24:	learn: 0.2788833	total: 374ms	remaining: 14.6s
25:	learn: 0.2728630	total: 395ms	remaining: 14.8s
26:	learn: 0.2667806	total: 412ms	remaining: 14.9s
27:	learn: 0.2603711	total: 425ms	remaining: 14.7s
28:	learn: 0.2549279	total: 435ms	remaining: 14.6s
29:	learn: 0.2500237	total: 445ms	remaining: 14.4s
30:	learn: 0.2461773	total: 461ms	r

In [33]:
print("K-fold 교차 검증 성능:")
print("평균 정확도:", sum(val_accuracy_list) / len(val_accuracy_list))
print("평균 정밀도:", sum(val_precision_list) / len(val_precision_list))
print("평균 재현율:", sum(val_recall_list) / len(val_recall_list))
print("평균 F1 스코어:", sum(val_f1_list) / len(val_f1_list))


K-fold 교차 검증 성능:
평균 정확도: 0.948447347239151
평균 정밀도: 0.8093760182469859
평균 재현율: 0.5656909446512244
평균 F1 스코어: 0.6645315933946425


In [35]:
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)


CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/libs/data/model_dataset_compatibility.cpp:81: At position 0 should be feature with name 유동비율(%) (found 순운전자본비율(%)).

In [ ]:
confusion_mat = confusion_matrix(y_test, y_test_pred)

# Test 데이터 평가 결과 출력
print("\nTest 데이터 평가:")
print("정확도:", test_accuracy)
print("정밀도:", test_precision)
print("재현율:", test_recall)
print("F1 스코어:", test_f1)

print("\n혼동 행렬:")
print(confusion_mat)


Test 데이터 평가:
정확도: 0.9265873015873016
정밀도: 0.47104247104247104
재현율: 0.5
F1 스코어: 0.485089463220676

혼동 행렬:
[[3147  137]
 [ 122  122]]
